In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import string
#import warnings; warnings.simplefilter('ignore')
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import random
import sys
import io

/home/tianaco/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [30]:
# Open the techno bands file
f = open('techno_bands.txt', "r")
 
# use readlines to read all lines in the file
lines = f.readlines()

In [31]:
len(lines)

5971

In [32]:
#right-stripping the end characters

lines2 = [s.rstrip('\n') for s in lines]

#deleting the extra annotations from the list

sep = ' ('
lines3 = [s.split(sep, 1)[0] for s in lines2]

#getting rid of other extra characters

lines3 = [s.replace('\\xc3\\', "") for s in lines3]

In [33]:
#the list is clean!

lines3[0:10]

['A-Trak',
 'Afrojack',
 'Alesso',
 'Erol Alkan',
 'Steve Aoki',
 'The Avener',
 'Avicii',
 'Danny Avila',
 'Benny Benassi',
 'Bingo Players']

In [12]:
#making the list into a dataframe

df = pd.DataFrame(lines3)

#changing the column's name

df.columns = [['DJs']]

#dropping duplicates

df = df.drop_duplicates()

#getting info

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5656 entries, 0 to 5970
Data columns (total 1 columns):
DJs    5656 non-null object
dtypes: object(1)
memory usage: 88.4+ KB


In [51]:
#exporting to csv for my records

df.to_csv("clean_DJ_names.csv")

In [52]:
#get the text ready

text = ' , '.join(lines3)

In [53]:
text

'A-Trak , Afrojack , Alesso , Erol Alkan , Steve Aoki , The Avener , Avicii , Danny Avila , Benny Benassi , Bingo Players , Blasterjaxx , The Bloody Beetroots , Bodyrox , Booka Shade , Boys Noize , Breakbot , Busy P , Felix Cartal , CFCF , The Chainsmokers , Chuckie , Clockwork , Cold Blank , Congorock , The Count & Sinden , Crookers , Dada Life , Daft Punk , John Dahlback , deadmau5 , Digitalism , Dimitri Vegas & Like Mike , Diplo , Dirty South , DJ Ajax , DJ Tocadisco , Duck Sauce , Fake Blood , Fedde le Grand , Feed Me , Felguk , Felix da Housecat , Dillon Francis , Joachim Garraud , Martin Garrix , Goldierocks , Green Velvet , David Guetta , Hard Rock Sofa , Hardwell , Calvin Harris , Icona Pop , Sebastian Ingrosso , Jack Beats , JDevil , Justice , Kavinsky , Kelis , Kill the Noise , The Knife , Knife Party , Krewella , Laidback Luke , Tommy Sunshine , Lazy Rich , Le Castle Vania , Crystal Castles , Tommy Lee , Com Truise , Lo-Fi-Fnk , Madeon , Major Lazer , Neon Indian , Malente ,

In [54]:
#looking at all of the characters and indices
#enumerating the characters

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 95


In [55]:
# cut the text in semi-redundant sequences of maxlen characters

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


nb sequences: 26017
Vectorization...


In [16]:
# build the model: a single LSTM

print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=200,
          callbacks=[print_callback])

Build model...
Epoch 1/200
25984/26019 [============================>.] - ETA: 0s - loss: 2.6804
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " Jellybean Benitez , Jindo , Jitwam , JK"
 Jellybean Benitez , Jindo , Jitwam , JKan , Danter , Den , Dane , Bane , Dashe , Salere , Bane , Banter , Dane , Bera , Dantere , Dene , Dana , Bantere , Dante , Dere , Dan , Dale , Dane , Dene , De , Bane , Dene , Dane , Dere , Dane , Dantere , Dan De , Bena , Dan , Dan Sane , Bantele , De Seane , Derale , Danter , Daleale , Dane , Dan De , Dane , De Seane , De Seane , Bane , Bane , Derele , Dene , Dane , Dan , Dan , The , Bane , De ,
----- diversity: 0.5
----- Generating with seed: " Jellybean Benitez , Jindo , Jitwam , JK"
 Jellybean Benitez , Jindo , Jitwam , JKaltere , Kantor , De Boan , Vansole , Beren , Dane , Ban Banse , Dale , Wanken , Bareserero , Daelehlon , Beno , Pele , Seale , DJ Bare , Deneereke , Mandan , Satoolel , De Soner , Mans De , Berarore ,

d , Datatraxx , Datax , Datis 5L , Datzming Dassait , Hulmang Duvp , Dupalo , Deatop Cars , Je.jaken Mint , AAPbar , Mis Vaadekery , Dzenjit , Cithe-tM Mort , WannelIJ Ganblas , And Heiny astica , Backlos , DanaboDge Bri Benrens , Eni-Pastins , witll , Dj Elln Scittton , Collvi Soddf , Oteteli Band , Parie And Bakau , Baelet Roscic Sandon , Naxber affeand , Poqkelbot , DJ Molomay , Dige Beno , MikerHeld , JactRacreze , Trica Fumlyy , Ne
----- diversity: 1.2
----- Generating with seed: "d , Datatraxx , Datax , Datis 5L , Datzm"
d , Datatraxx , Datax , Datis 5L , Datzmy jacd , Hul Kersrobl Rodrecs , WrKchneing In DS JaarSsymiclkecuee , Drchab Fatoiavan , Bisscennedinatrez , Tenenn Kunzldstev , Mikogidax , Dnimphive Dast , Big Rumbinn Tomanc , Recil-orn , Boab , Zell Domatamh , Preln Bech lata , Kanpan Sub Fade , Haoruik Sin Saulxeisrast , Sabndyeg Oruchiedly , Kast Fiaso , Krogl.D.D. , ErmsthI Benz , Dscridlax Donzscazs , Dapend Bils Fractex , Draftrox , Deerg 
26019/26019 [=============

, Bolivar , Boody B , Bookworms , Boris & , Funnion , Feanter , Gerrin , Funnie Bann , Feander , Feanters , Geore , Feander , Feanters & Deeper , Feansen , Feand , Feansen , Feanters , Feand , Fean Berger , Fearse , Fith Sparker , Fill , Fill Charce , Fill Frooderer , Fill , Fith Sparric , Giool Seans , Feander , Geres , Fearters , Feanters , Feander , Fill Heeder , Feander , Feanters , Feather , Fean Schers , Feand , Feath Sears , Fear
----- diversity: 0.5
----- Generating with seed: ", Bolivar , Boody B , Bookworms , Boris "
, Bolivar , Boody B , Bookworms , Boris , Kiv , Flaush Mould , Featie Seecrion , Fill A Tirge , Time Garss , The Fithers Jace , The Stares , The Willo , The Grosh Sterma , Traces , The Kint Dobberg , The Beat Head , The Tist , The Stare , The Horder , The Purtere , The Spafe & Jahn , Black Manting , Blay Heeder , Factrastion Barter , Fater Deerer , Bassion , Bassian , Baskura , Bassion , Basside , Fass Cinger , Fassmar , Barcital , Tand
----- diversity: 1.0
-----

/home/tianaco/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


de , DJ Gliths , DrujlOp , Lovs Howdens , WitiaKou , Unkstrame , itK Sange , Ricya , Jn Ock , Alar Dodlmans , ZadSrug , Estalh Angert , Uroh Fripkker , Vallfaxwno Tho Beskmann , Jussmack , w
26019/26019 [==============================] - 53s 2ms/step - loss: 1.3048
Epoch 13/200
25984/26019 [============================>.] - ETA: 0s - loss: 1.2403
----- Generating text after Epoch: 12
----- diversity: 0.2
----- Generating with seed: "e , Mr C , Mr Scruff , Mr. G , Mr. Ties "
e , Mr C , Mr Scruff , Mr. G , Mr. Ties , Mr. Dunk , Mr. Gerrin , Mr. , Mrey Flau , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr. , Mr
----- diversity: 0.5
----- Generating with seed: "e , Mr C , Mr Scruff , Mr. G ,

...

 T.A.T.u. , Techno Animal , Techno Twins , Sephers , Selzostion , Selversky , Secrez , Specker , Sepice & The Woster , Sweeted Stameh , Secaciz , Sepill , Secret Benfect , Sever , Serge , SevinyDad 2 , Penzy , Peller Benz , Pezzi , Plantono , Plaiant , PlattSomm , Prand Conna , Prodes & Cinsen , Bieta , Bitz Snoff , Blip Gegst , Glyands , Black Mate Jannode , Gischiger , Grysso Seanden , Gusura Bena , Beyrab , Bey , Beet , Bitrom , Bita
----- diversity: 1.0
----- Generating with seed: " T.A.T.u. , Techno Animal , Techno Twins"
 T.A.T.u. , Techno Animal , Techno Twins , Sevolus , Seggffffoodri , SemzinqlecJinn , Slaith Od Sound , Slyt Syikon Buga , Symer , Smmehroo , Sony Mayous , Serge Rechide , Sevin Dancen , Sevonurfe , SeveUn.Ho , Sony Malos , Sony Malog , Sony Malos , Sony Mayos , Somad , Sobert Dorce , Somid , Son Coez , Sonanus , Somad , Sorte Tanseor , Sorterans , Sound Manter , Zeromy H. , Unkor Lotri , Unkor Hope , Undrice , Intarg , Imnor Cascliia ,
----- diversity: 1.2
-----

n Ravelo , Adriano Clemente , Aear , Aesike , Awdleak , Specker Dairect , Stereo Oyd , Thie MK , Thiqsa , Twey Roby Stech , Swinnra Canter , Sticky , Skin , Sliption , SMr. , SMD , AngalAN , Sacker A , Sandrie Sound Sythming , Savante , Sather Anant , Sander va , Sallormance , Savante , Sanduer vo Zahis , T&SbVo Timers , Superstam , SUinge , Suzaqurta , SQaderyt Seiffore , Sceet Tine , Sperr , Sperg , Sperk , Sperg Wallmann , Stre Age ,
----- diversity: 1.0
----- Generating with seed: "n Ravelo , Adriano Clemente , Aear , Aes"
n Ravelo , Adriano Clemente , Aear , Aesini Gark , Ai &Srikk , Alid Roover , AlmTon Va Vincent Tapp , Vogny Tomal , Vorgin , Vogvathima , Volvassin Jastonded , Monuento , More Falgars , Votorice , Vrisson , Vrighthian , Morical D , Mimdes Willof , Mlubluca , MDJ Growse , Missoodila , Mischiss , Machine Modentip , Mr. Grow Nitala , Mr. OrabilDf , Mr. Grinces , Mr. Nithey , Mourta , MDG O-Benk , MaA2riike , Anter , Anti , Ante , Ante , An
----- diversity: 1.2
-----

 Kangding Ray , Karen Gwyer , Karenn , Kampootic , Kamerker , Kasser , Kasser , KasiPig , Kan Romer , Karen Garow , Kamar , Kasser , Kasters , Kamrie e , Karen Modective , Kamer Wallever , Theal Gessy , The Knofects , Treyo Supery , Trafech , Kramecters , Krama , KJach , Kike Willew , Kiuls , Kugnn. , Komplee , Kon Moric , Knoship , Kovan Pinster , Kun Smith , Kompoter , Konik , Konko Seirie , Kenno , Kemin Sound Sound Sykeomica , Henni
----- diversity: 1.0
----- Generating with seed: " Kangding Ray , Karen Gwyer , Karenn , K"
 Kangding Ray , Karen Gwyer , Karenn , Kamine , Kany Ed , The haftorc , The May Bent , Ste Dug , Sim Teran , SiJure Glaer , Slouy , Sleeize Tone , SNiZk & Checri , Smmejae , Suzabady , Skin , SMC K & Hunder Rey , SychiWa , Sacria , Sandet Stene , Saschy , Sasife Tick , Salmers Oher zame , Tanbole , Telepiould , Keniva Gros Pip , He Re& y Gurner , Hei , Hiftro An , Gio Garry , Grot Seee , Gret Seee , Gregs , Greet Bena , Gis Mary , Filu 
----- diversity: 1.2
-----

a Lapsin , Sansui , Sante , Santiago Lect , Sande Sashem , Sarter x , Scoea Farkstronds , ScaUore Cristet , Space Back , Speaker , Spacimag , Speachin Bathers , Space Bumaglasse , Space Bug , Speaz , Sperg , Sperg , Spergracert , Speakeaz , Shleet The Shaus , SiD-Sight , Supvem , Super Filtets , Superstament , Superstamento , Superstamonn , Son Modey , Sonaste , Sony , Sony , Sony , Sony , Sony , Sony , Sony , Sony , Sony , Sony , Sony 
----- diversity: 1.0
----- Generating with seed: "a Lapsin , Sansui , Sante , Santiago Lec"
a Lapsin , Sansui , Sante , Santiago Lect , Scoea Sexite , Shautible D , Sparala , Spaclia , Samal  , Samin Dress , Sandrum , Sande Sunneming , Sander thies , Sande Synfther , Savads , Scot Jefella , Sashers .tabo , The Rachiffey , The Sunfgyz , T'yfife , Tleil & Supgerm , Turryss , Tiliophike Vassick , Thiqsahin Crosserme, , The Buck , The Sunser , The Sunsender , Thik Carmage , Knambot , Kovop Wote , Koniger Spede , Kylwije , Kewer , 
----- diversity: 1.2
-----

, Highflesh , Highgrade Disharmonic Orchis , The Hunders , The Haush , The Houshwark Beanden , The Hanandex , The Handy , The Hausheatra , The Krafhers , The Hanaos , The Ghove Groshis , Grey Gash , Gren Swingtes , Fritze Simkre , Grot Perrian , Frither Shandeds , Grot Reversin , Grooveurs , Gree , Gree , Gree , Gree , Gregs Jamesuen , Gret Whermboude , Giscor x Fengon , Gors Mes , Donifarge , Dunky , Dunky , Dunkat Phomitce , Dunkysjaz
----- diversity: 1.0
----- Generating with seed: ", Highflesh , Highgrade Disharmonic Orch"
, Highflesh , Highgrade Disharmonic Orchis , The Hannebruth , The Druiria A , Erbor , Eyo Ed , Eyofy , Braik Sum , Bach Buth , Baik Kaby , PEambi , Polheda , Poodwet , Porter at Pustiva , Pnyvan , Pornica Roon Q , Portis Papher , Portces Capeolect , Polye Dewill , Pollashimmb , Polyatheat Of Nater , Pnaker , Poora Shana , Pelie an Dewrither , Peger Stavewae , Percea , Perge Calarh Cover , Pelephann , Pers & Sharom , Perret Art , Per.Anx
----- diversity: 1.2
-----

### I ran this a few times with different # of maxlen characters, epochs and different seperators for the DJ names. I got better results with commas as a separator. I got better results with 40 maxlen characters than 10 and 20. I also got great results around 20-60 Epochs and marginal returns after that. 

### I basically looked at the output and scanned the list for RNN-generated DJ names that I liked, thought were funny, or sounded like a real DJ name. They are listed below

# the long list of neural net dj names

T:X Lise, rosS, Pin Mas, Teris, Subin, DJ Priss, DJ DJ, DJ Spint, Solaman, The Gour, Wincas, toe Cock, C!n Kasbri, The Harder, The Sinne, Mars Sun, Sharter, TWo Snam, Dastic Mon, Maris Olen, Dalia Saster, Masom Desta, Deana, Nassholl, Danty Dader, Mart & Darc, Mikogidax, Bloude, The Horder, Gorder of Sound, The Tree, Tilo, Houde Sounds, Soren, Herol, EKchoon, Bassini, The Ham, The Wigge, Tedy & 8h, The Vikk, Krate, Black Divial, The Kitter, Greg Su, Black Mario, Tom Sound, Bloy und Stats, Dank, Bassmount, Bassy Tocky, DJ Discolice, Mono, Dirty Faggs, Arty Rober, Ante Cunt, The Chers, Red of Julled, Discry, Lunal Bey, Zie & Krrks, Sandstan, Solentice, Staric Age, Panzie, Starion, The DOd, DreamMe, Yike Flood, TUNR, The Mood, Mr. Grey, Stary in Space, Clarial, Crapers, Hunny, Wi0kiam, Wonkerflay, KuC, Peanuster, The Fockwerds, Frandmas, Roboski, Acid Poopa, The Handelle Fiight, Blackwirs, Erasz, John Mad, The Puns, GOD Asela, Chariom, San U, Hifter Div, Mada Bee, DJ Snitto, Dibo, Hey thing, ju Creed, Blaz, Blund Mann, Backweat, Ant, Fockwark, Bary Loys, Baansonik, DiscoKasem, Alia Pank, My Rainetoon, vopektip, Roborky, RlROI, Eg'zo, The Honsy, Kian Arteron, Diskmurt, Discomess, FucksiT, Disonowar, Miss Modatick, Mr.S.OD, Mist Tab, Mr. Oe, MJ Trip, Valiatope, Pissoow, Hilo x Disco, Mikiat, The Groobly, The Ghostick, Hard Tie, Kario, Sonidsmic, Black C-Ich, Wolfgastic, Bodysmic, Less Moor, Qt Jamsue, Horsemann, The Gross, Ellen Lush, Slowblu, The Croney, Swaims, The Plama, Gloo, Gorisseka, CrookHop, Borik P, Esconia, Mr. Gringloy, SIN, W-ZAg, Nomistiss, Shiva Fag, Beetrabit, Bean Walhine, b.E.A, Zosami, INDurest, And:raiders, Dicklopes, DJ Rolex, DJ Snitz, The Dave, DuDKE, Audiopick, Art Cock, Moodle, Saaksex, DJ XTip, Black Matt Cantine, Black Black, Black Dissonis, Mr. Glayo, Enfofox, Di Luma, Digitalic Modim, ManaHand,  Discrostar, Ameurika, Dub Sing, Hunky, Mintey Fat, Deep Warter, MAGGip, Mist Sounds, Math Mash, ShaKy, time Worky, vie Fuckers, Diskothron, Dreambay, Kapo Tocky, The Boote, Toie Fu, The Fatther, Devin Dunk, Oscopes, Istara, LCM.On, Dr.Spark, Booka Snex, The Handy Turk, Antis Kinnerlot, Mr. Grocker, Mr. Oula, Daady Triss, Lafwines, D NouSh, DOLFu, Miss Egg, The Sub ChicKTote, Gregsturd Lust, Grandsma, Discawulla, WONDuz�, Kroote, Penius, Martin Disco, Discave, Trax Swien, THRIK, Vialection Dagutser, The -zaz, Electrovall, Deleklockwarke, Turdd, Miriom, DJ TerFF, Daady Tick, DJ MoCk, Edio Fat River, King Gut, Dragon Black, Lex Lunz, Dunny Bunks, Loop Belline, Hey Dick, Boo Maggin, Mike Disco, Dirty Harder, DJ Sponky, Machinskoople, Lost King, Black tarick Tool, Super Filthy, Speerrm, ScreamKnex, Superythm, The Surfer, The Hack3rdd, The Sucky, Tripe witch, The Handy, The Haushmasker, Cause of Kaze, Callfood, The Sucker, 2zig, Volvas, LeeSpong Dub, Wongk, Go Go D Doepals, Lovagie, Luzy Love, House of Boby, Horse of Salent, The Bong, The Sucks, ScaRe, Thing Rob, Tenant, Superceps, Superstich, Symug, Lonas, Loverything Vine, Lisey if Cake, Lisbo, Lift Smoke, Liver & Hundey, Like Sifter, Lensewarkstreel, Lens Modican, Snychonz, Syn-loo, The Duna, The.2mand, State Mountine Tonga, Skary, Blockwerk, Chaist, Black Dre W, The New lakia, Steal vit, Play Sound, Mista Rodger, Manjacio, MamaciEl, Kike Supermeds, Mono Fathers Vomato, Scat Crint, Nolver the moothming, No and Nerolor, Cart Love, Spahming, SpearDopplectivio,  Goodles Benz, Green Facks, Visto Modique, Misso, No OK, AirLuk, Altwill, Al Weak, Mystape, U.Su, Che Bears, The Rico, H. Black, Kneck, Mark Soul, Mint Rog, Monoellective, The Stepes DJ, Super8Skz, Supercoy, Matte Vo, Miss Mad, Myste an Valinas, MisKabali, Super Bug, DJ the Don, DJ Man, Coldionic Synth, Mr. Nico, Mr. Bo, Mr. Glax, Horsemann, Homi-en, Nudio, Friendss, Play Sounds, Phat Room, The Blacksta, The Rapes, The Exx, Dunky, Salvar, Pan In Soul, Von MoQ, DJ Silke e, Wong Dope, Pan Lamb, PartiKinck, Rod Soul, Tond Sonner, Blackist 30s, &SThen, Hinds Wall., Cod Crack, DJ Stunkers, DJ Thing, Slik, Spancoider, ziffe, Felican Fight, Conjo., DJ H, Aman Haus Of Yie, Arano Collettive, Ana Bermann, Higorabit, Time Alice, Rackicar, The Kuk, Rockit Chorder, Jon Mar, DJ STing, DJ Spinna, The Panda Baster, Golde Dune, Phat Mastorcon, The Krow, Speaky, Super Fight, Bongo Whorge, Bar Maria, Bad Mary Lige, Barber Chike, Ally King, Discopit Dorisce, Dearcop, Beat Spaco, Audia, B. Dunkon Cida, Antiloop, Allectione, Alix Cita, The Kingles, Trip loop, Wolfa Fund, Love Lock, Love Laci, Wolfgack, Mr.Mand, Miroslavion, Goldez, Gow Age, Eis Al The Nike, Super Cog, Super Chum, Soulwest, Sambot, DaDas, Cause 3000, Horsey Lopp, Pag-Man umpire, Beanes Snake, Blackwax, Black Ming, The King Lub, The Mush Mashaus, Betal, Primaniks, Templast, zon, Tymotica, Dilo, The Mars, Mishi, DJ Jug, Machine Womamaponaka, Hoko Go, LAq, Robostic, Caran Sound, Affectrick, Amb-Lovo, Tom Osea, The Sun J, The Ming Rash, The Housemass, The Sun Sit, The Dub Sound on Sound, Shit Sea, Black Wallekers, Black Moune, Black Machine, Charmer, Cale Eks, BaXdy Rifi, AMac, Deetronic, Cliptals, Farter Spiper, Crunti, Clarion, Aqua Bager, Amotrick, Aqua Bassil, Bash K, Barters, Aran Agia, Aqua Baser, LASTect, Lost Priver, Boy Tactric, Love & Mauna, Valachia, Kooke, Zakan, The Kom Goos & Chongat, Joan x, TS St0f, Don Trice, tep.o, Ray Ox, Hary Loud, The Gryst of Dirta, Shakely p, Sparte Sun, N.A.R.A., Ezzie, Emik Samie, Enter Sound, Interst, Ellee Bark, Bender, Bearian, Elee the Heat, Marcine Go, Mink Pade, Miss Moss, Midiscation, Miss Mofiek, Mr. Onerate, Marcon Bite, Clivion Fumer Propelleker, The Peag, Sandelic, Paal Kink, Coppari, Caper & Mal, Bearfiella, Barman Sachs, Ben-satorcella, Fat Kim Orcenco, SONA, Claudian Spaces, The Space, Horsemellic, Black & Couth, Blackers, Black Crifte, Black Machiniolo, Bearmann, The Mone, Trickle z, Black Stare, Black Divins, LaLuK, Loha, Datra Kasmiror, Divi-t, Dubbotabt, Amesion Space, La Rocker, Bassive, Ras Kiddiz, Ramik Rok, Hardsooother, M.A.N.De, DreamMantin, Drum Punk, Bassivers, Booka Sam, Booka Rock, NowMor, Nict Math, Spairal, MOmik, will Vollman, Jumanna the H:ryse, Krux, Noka Dinky, Vigama, VigDasoric, Victor Chang, Victor Chance, Bassivers, Bassivers, Creure Drev, Crupal, Cris of Viver, Crupalice, Bitatam, Totol Owers, Discrotic, Discooos, Bass Hardy, Audiferon, Acid Pessimique, Bassbookers, Manderilly, Infora Play, Dirty Ram, DJ Salute, Darios, Rolett, Sudio Jony, Hunnith, Sorian Of 300, Clariolain, Super Moop, Super Moves, LA Fires, Frilly Pot, Acid Bomple, Block Spack Mik, Benn 20, Los Movele, Love Terag, Carverfin, Hallal Butz, The Gills, underson, Rory St Jous, The Core, The Code, The Neuron, Techit, The Croove, The Ontx, GolylxDi, Mocce Handersen, Shanks & Bitr, Ross Oh Stone, Borotice Borker, Cousspecky, Mordekt, Noty The Hookers, Lando, Jonn Glaye, DJ Twing, Dassion Dave, Minispeep, Eleccothinn, Jagur Dickson, Miss Coda, Estrolgo K, Octanne Jaw, Blanka Shitracts, Blanker Bunk, The Bird, Double Dragon Karner, Carlo Slopp, Black Syster, Atspectopez, Tinkz Ramineen, TitDuc, The Stank, The Chatelle, Hear, Anny Clack, JJ Pie, Sub Bobler, Ly Clawer, Airy Dean, The Chailers, DJ Starker, DJ Black, DJ Vew, The Tic Tim Hillentrakk, The Coder Nights, Colotobod, Falt Aire, Osio Hanal, Sant Aids, Azaidius of Lud, Soundsyster, Deep Mutter, Deep Mindor, Iron Hanke, Konny Of Lick, Kon Jansen, Konnnand, TechnolaHax, Duckystem, MonoLot, Hoy Slave, Soundsellen, Son Kinder, Souza de Dird, Sinka Pit, Erka Deep, Electroches, Eleccer Duke, The Marchine and Aristice, Avealo Luspice, Bindelly Shr., Bingoong, The Favers, Acrack, Luda Wivid, Repht, Slim & Ead, Beat Box, Sonny Ride, Tommosi, Gloria Fore

In [1]:
#put DJ names into a list

dj_list = ["T:X Lise, rosS, Pin Mas, Teris, Subin, DJ Priss, DJ DJ, DJ Spint, Solaman, The Gour, Wincas, toe Cock, C!n Kasbri, The Harder, The Sinne, Mars Sun, Sharter, TWo Snam, Dastic Mon, Maris Olen, Dalia Saster, Masom Desta, Deana, Nassholl, Danty Dader, Mart & Darc, Mikogidax, Bloude, The Horder, Gorder of Sound, The Tree, Tilo, Houde Sounds, Soren, Herol, EKchoon, Bassini, The Ham, The Wigge, Tedy & 8h, The Vikk, Krate, Black Divial, The Kitter, Greg Su, Black Mario, Tom Sound, Bloy und Stats, Dank, Bassmount, Bassy Tocky, DJ Discolice, Mono, Dirty Faggs, Arty Rober, Ante Cunt, The Chers, Red of Julled, Discry, Lunal Bey, Zie & Krrks, Sandstan, Solentice, Staric Age, Panzie, Starion, The DOd, DreamMe, Yike Flood, TUNR, The Mood, Mr. Grey, Stary in Space, Clarial, Crapers, Hunny, Wi0kiam, Wonkerflay, KuC, Peanuster, The Fockwerds, Frandmas, Roboski, Acid Poopa, The Handelle Fiight, Blackwirs, Erasz, John Mad, The Puns, GOD Asela, Chariom, San U, Hifter Div, Mada Bee, DJ Snitto, Dibo, Hey thing, ju Creed, Blaz, Blund Mann, Backweat, Ant, Fockwark, Bary Loys, Baansonik, DiscoKasem, Alia Pank, My Rainetoon, vopektip, Roborky, RlROI, Eg'zo, The Honsy, Kian Arteron, Diskmurt, Discomess, FucksiT, Disonowar, Miss Modatick, Mr.S.OD, Mist Tab, Mr. Oe, MJ Trip, Valiatope, Pissoow, Hilo x Disco, Mikiat, The Groobly, The Ghostick, Hard Tie, Kario, Sonidsmic, Black C-Ich, Wolfgastic, Bodysmic, Less Moor, Qt Jamsue, Horsemann, The Gross, Ellen Lush, Slowblu, The Croney, Swaims, The Plama, Gloo, Gorisseka, CrookHop, Borik P, Esconia, Mr. Gringloy, SIN, W-ZAg, Nomistiss, Shiva Fag, Beetrabit, Bean Walhine, b.E.A, Zosami, INDurest, And:raiders, Dicklopes, DJ Rolex, DJ Snitz, The Dave, DuDKE, Audiopick, Art Cock, Moodle, Saaksex, DJ XTip, Black Matt Cantine, Black Black, Black Dissonis, Mr. Glayo, Enfofox, Di Luma, Digitalic Modim, ManaHand, Discrostar, Ameurika, Dub Sing, Hunky, Mintey Fat, Deep Warter, MAGGip, Mist Sounds, Math Mash, ShaKy, time Worky, vie Fuckers, Diskothron, Dreambay, Kapo Tocky, The Boote, Toie Fu, The Fatther, Devin Dunk, Oscopes, Istara, LCM.On, Dr.Spark, Booka Snex, The Handy Turk, Antis Kinnerlot, Mr. Grocker, Mr. Oula, Daady Triss, Lafwines, D NouSh, DOLFu, Miss Egg, The Sub ChicKTote, Gregsturd Lust, Grandsma, Discawulla, WONDuz�, Kroote, Penius, Martin Disco, Discave, Trax Swien, THRIK, Vialection Dagutser, The -zaz, Electrovall, Deleklockwarke, Turdd, Miriom, DJ TerFF, Daady Tick, DJ MoCk, Edio Fat River, King Gut, Dragon Black, Lex Lunz, Dunny Bunks, Loop Belline, Hey Dick, Boo Maggin, Mike Disco, Dirty Harder, DJ Sponky, Machinskoople, Lost King, Black tarick Tool, Super Filthy, Speerrm, ScreamKnex, Superythm, The Surfer, The Hack3rdd, The Sucky, Tripe witch, The Handy, The Haushmasker, Cause of Kaze, Callfood, The Sucker, 2zig, Volvas, LeeSpong Dub, Wongk, Go Go D Doepals, Lovagie, Luzy Love, House of Boby, Horse of Salent, The Bong, The Sucks, ScaRe, Thing Rob, Tenant, Superceps, Superstich, Symug, Lonas, Loverything Vine, Lisey if Cake, Lisbo, Lift Smoke, Liver & Hundey, Like Sifter, Lensewarkstreel, Lens Modican, Snychonz, Syn-loo, The Duna, The.2mand, State Mountine Tonga, Skary, Blockwerk, Chaist, Black Dre W, The New lakia, Steal vit, Play Sound, Mista Rodger, Manjacio, MamaciEl, Kike Supermeds, Mono Fathers Vomato, Scat Crint, Nolver the moothming, No and Nerolor, Cart Love, Spahming, SpearDopplectivio, Goodles Benz, Green Facks, Visto Modique, Misso, No OK, AirLuk, Altwill, Al Weak, Mystape, U.Su, Che Bears, The Rico, H. Black, Kneck, Mark Soul, Mint Rog, Monoellective, The Stepes DJ, Super8Skz, Supercoy, Matte Vo, Miss Mad, Myste an Valinas, MisKabali, Super Bug, DJ the Don, DJ Man, Coldionic Synth, Mr. Nico, Mr. Bo, Mr. Glax, Horsemann, Homi-en, Nudio, Friendss, Play Sounds, Phat Room, The Blacksta, The Rapes, The Exx, Dunky, Salvar, Pan In Soul, Von MoQ, DJ Silke e, Wong Dope, Pan Lamb, PartiKinck, Rod Soul, Tond Sonner, Blackist 30s, &SThen, Hinds Wall., Cod Crack, DJ Stunkers, DJ Thing, Slik, Spancoider, ziffe, Felican Fight, Conjo., DJ H, Aman Haus Of Yie, Arano Collettive, Ana Bermann, Higorabit, Time Alice, Rackicar, The Kuk, Rockit Chorder, Jon Mar, DJ STing, DJ Spinna, The Panda Baster, Golde Dune, Phat Mastorcon, The Krow, Speaky, Super Fight, Bongo Whorge, Bar Maria, Bad Mary Lige, Barber Chike, Ally King, Discopit Dorisce, Dearcop, Beat Spaco, Audia, B. Dunkon Cida, Antiloop, Allectione, Alix Cita, The Kingles, Trip loop, Wolfa Fund, Love Lock, Love Laci, Wolfgack, Mr.Mand, Miroslavion, Goldez, Gow Age, Eis Al The Nike, Super Cog, Super Chum, Soulwest, Sambot, DaDas, Cause 3000, Horsey Lopp, Pag-Man umpire, Beanes Snake, Blackwax, Black Ming, The King Lub, The Mush Mashaus, Betal, Primaniks, Templast, zon, Tymotica, Dilo, The Mars, Mishi, DJ Jug, Machine Womamaponaka, Hoko Go, LAq, Robostic, Caran Sound, Affectrick, Amb-Lovo, Tom Osea, The Sun J, The Ming Rash, The Housemass, The Dub Sound on Sound, Shit Sea, Black Wallekers, Black Moune, Black Machine, Charmer, Cale Eks, BaXdy Rifi, AMac, Deetronic, Cliptals, Farter Spiper, Crunti, Clarion, Aqua Bager, Amotrick, Aqua Bassil, Bash K, Barters, Aran Agia, Aqua Baser, LASTect, Lost Priver, Boy Tactric, Love & Mauna, Valachia, Kooke, Zakan, The Kom Goos & Chongat, Joan x, TS St0f, Don Trice, tep.o, Ray Ox, Hary Loud, The Gryst of Dirta, Shakely p, Sparte Sun, N.A.R.A., Ezzie, Emik Samie, Enter Sound, Interst, Ellee Bark, Bender, Bearian, Elee the Heat, Marcine Go, Mink Pade, Miss Moss, Midiscation, Miss Mofiek, Mr. Onerate, Marcon Bite, Clivion Fumer Propelleker, The Peag, Sandelic, Paal Kink, Coppari, Caper & Mal, Bearfiella, Barman Sachs, Ben-satorcella, Fat Kim Orcenco, SONA, Claudian Spaces, The Space, Horsemellic, Black & Couth, Blackers, Black Crifte, Black Machiniolo, Bearmann, The Mone, Trickle z, Black Stare, Black Divins, LaLuK, Loha, Datra Kasmiror, Divi-t, Dubbotabt, Amesion Space, La Rocker, Bassive, Ras Kiddiz, Ramik Rok, Hardsooother, M.A.N.De, DreamMantin, Drum Punk, Bassivers, Booka Sam, Booka Rock, NowMor, Nict Math, Spairal, MOmik, will Vollman, Jumanna the H:ryse, Krux, Noka Dinky, Vigama, VigDasoric, Victor Chang, Victor Chance, Bassivers, Bassivers, Creure Drev, Cris of Viver, Crupalice, Bitatam, Totol Owers, Discrotic, Discooos, Bass Hardy, Audiferon, Acid Pessimique, Bassbookers, Manderilly, Infora Play, Dirty Ram, DJ Salute, Darios, Rolett, Sudio Jony, Hunnith, Sorian Of 300, Clariolain, Super Moop, Super Moves, LA Fires, Frilly Pot, Acid Bomple, Los Movele, Carverfin, Hallal Butz, The Gills, underson, Rory St Jous, The Core, The Code, The Neuron, Techit, The Croove, The Ontx, GolylxDi, Mocce Handersen, Shanks & Bitr, Ross Oh Stone, Borotice Borker, Cousspecky, Mordekt, Noty The Hookers, Lando, Jonn Glaye, DJ Twing, Dassion Dave, Minispeep, Eleccothinn, Jagur Dickson, Miss Coda, Estrolgo K, Octanne Jaw, Blanka Shitracts, Blanker Bunk, The Bird, Double Dragon Karner, Carlo Slopp, Black Syster, Atspectopez, Tinkz Ramineen, TitDuc, The Stank, The Chatelle, Hear, Anny Clack, JJ Pie, Sub Bobler, Ly Clawer, Airy Dean, The Chailers, DJ Starker, DJ Black, DJ Vew, The Tic Tim Hillentrakk, The Coder Nights, Colotobod, Falt Aire, Osio Hanal, Sant Aids, Azaidius of Lud, Soundsyster, Deep Mutter, Deep Mindor, Iron Hanke, Konny Of Lick, Kon Jansen, Konnnand, TechnolaHax, Duckystem, MonoLot, Hoy Slave, Soundsellen, Son Kinder, Souza de Dird, Sinka Pit, Erka Deep, Electroches, Eleccer Duke, The Marchine and Aristice, Avealo Luspice, Bindelly Shr., Bingoong, The Favers, Acrack, Luda Wivid, Repht, Slim & Ead, Beat Box, Sonny Ride, Tommosi, Gloria Fore"] 

In [2]:
# fixing the list so that each DJ name displays a a character

dj_list2 = []
for word in dj_list:
    word = word.split(", ")
    dj_list2.extend(word)
    dj_list2

In [3]:
#we have 666 DJ names

len(dj_list2)

666

In [4]:
#eliminating duplicates

dj_list2 = set(dj_list2)

In [118]:
#only 3 duplicates in neural network list

len(dj_list2)

663

In [13]:
#only 6 names in my edit from the original DJ list scrape 
#were generated by the neural network that are in the original list

duplicates = set(lines3).intersection(dj_list2)
duplicates = list(duplicates)
duplicates

['DJ Spinna', 'Antiloop', 'Tymotica', 'The Mood', 'Mono', 'Tenant']

In [15]:
#removing duplicates from the neural net dj names list

dj_names = [x for x in dj_list2 if x not in duplicates]

In [16]:
len(dj_names)

657

In [18]:
#putting the unduplicated RNN DJ names into a Dataframe so I can examine by attribute

DJ = pd.DataFrame(dj_names)
DJ.columns = [['DJs']]

In [19]:
DJ.head()

,DJs
0,No and Nerolor
1,Mada Bee
2,Enter Sound
3,Mart & Darc
4,D NouSh


In [20]:
#searching for strings

def name_count(x):
    g = DJ[DJ['DJs'].str.contains(x)==True]
    return len(g['DJs'])
    return g

In [22]:
#another function to return these name clusters

def find_sim(name):
    print(name_count(name))
    return DJ[DJ['DJs'].str.contains(name)==True]

In [23]:
#I noticed a lot of "Black" DJ Names

name = "black|Black|blak|Blak"
find_sim(name)

26


,DJs
40,Blackers
63,Black Dissonis
73,Black Ming
97,Black Stare
137,Black & Couth
180,Black Dre W
236,Dragon Black
273,DJ Black
285,Black C-Ich
315,Black Machine


In [24]:
#and a lot of DJ names with DJ in it

name = "DJ"
find_sim(name)

25


,DJs
20,DJ Snitto
90,DJ Priss
141,DJ Vew
156,The Stepes DJ
175,DJ STing
248,DJ XTip
273,DJ Black
286,DJ Snitz
316,DJ H
329,DJ TerFF


In [215]:
#lots of animal names

name = "bear|Bear|Horse|horse|Panda|panda|Ox|Fox|fox|cod|fish|Cod |fin|Snake|snake|Dragon|rabit|wolf|Wolf|lamb|Lamb|Mink|mink|Bark|bark|Duck|duck"
find_sim(name)

25


,DJs
75,Wolfa Fund
83,Wolfgastic
129,Double Dragon Karner
133,Mink Pade
187,The Panda Baster
226,Pan Lamb
236,Dragon Black
255,Carverfin
266,Horse of Salent
268,Horsemellic


In [177]:
#the RNN definitely picked up on "disc" and "disco"

name = "Disc|disc"
find_sim(name)

14


,DJs
95,Discawulla
174,Martin Disco
198,Mike Disco
217,Discomess
247,Discave
253,Midiscation
287,Discooos
435,Hilo x Disco
454,DJ Discolice
463,Discry


In [25]:
#and sound

name = "sound|Sound"
find_sim(name)

11


,DJs
2,Enter Sound
128,Caran Sound
188,Gorder of Sound
214,Mist Sounds
269,Soundsyster
270,Tom Sound
278,The Dub Sound on Sound
398,Houde Sounds
556,Play Sound
587,Soundsellen


In [195]:
#lots of miss and mister names

name = "miss|Miss|Mr"
find_sim(name)

19


,DJs
16,Mr. Nico
53,Mr. Oula
78,Mr. Grocker
113,Miss Mad
153,Mr. Glax
199,Miss Egg
265,Mr. Onerate
305,Mr. Oe
313,Mr. Glayo
349,Mr. Bo


In [136]:
#alternative characters

name = "0|1|2|3|4|5|6|7|8|9|!|@|#|%"
find_sim(name)

11


,DJs
51,Sorian Of 300
96,C!n Kasbri
151,TS St0f
167,Tedy & 8h
319,2zig
437,The Hack3rdd
464,Cause 3000
509,Super8Skz
544,Wi0kiam
570,The.2mand


In [137]:
# ____ & _____ names are pretty common among DJs and the RNN picked up on that
#there were a ton of these in the output but they weren't particularly interesting so I didn't include many
#in my edit

name = "&"
find_sim(name)

11


,DJs
3,Mart & Darc
11,Slim & Ead
98,&SThen
137,Black & Couth
147,Love & Mauna
167,Tedy & 8h
526,Shanks & Bitr
584,Liver & Hundey
608,The Kom Goos & Chongat
610,Caper & Mal


In [222]:
#family names were common

name = "sister|Sister|Syster|syster|randma|Dad|dad|Sis|sis|Bro|bro|Fatther|Daady|Grand"
find_sim(name)

8


,DJs
41,Daady Triss
72,Frandmas
210,The Fatther
269,Soundsyster
277,Danty Dader
448,Daady Tick
592,Black Syster
639,Grandsma


### Future steps include running the text on a more complex neural network, and trying different approaches with the text organization and saving RNN epoch network weights in checkpoint files and using those parameters as a basis for more text generation. But for now I enjoyed watching the neural network run each epoch with different seeds and diversity.

## I'm currently recruiting friends in DJ/techno culture to help me out with a curation of the best, funniest and weirdest names. Stay posted for that content. Read on if you aren't offended.

# CONTENT WARNING:
# The content below can be considered "Adult", Rude and Offensive.
# If you are offended by sex and drug references and 4-letter words, do not read any further.

In [29]:
#drug names & slang

name = "LSD|coke|Coke|Crack|Weed|Acid|acid|bong|Bong|Dank|dank|Dope|dope"
find_sim(name)

8


,DJs
32,Bongo Whorge
358,Acid Poopa
359,The Bong
367,Acid Bomple
414,Dank
456,Acid Pessimique
609,Cod Crack
617,Wong Dope


In [216]:
#dirty and offensive words

name = "Fuck|Fock|fuck|fock|Fack|fack|Sex|sex|Dick|dick|Ass|Butt|butt|lick|Lick|But|Sperm|Speerrm|Cunt|cunt|Cock|cock|Piss|piss|fag|Fag|shit|Shit|penius|Penius|Penis|Kike|Scat|assho|turd|Turd|suck|Suck|Kink|Kinck|Fart"
find_sim(name)

32


,DJs
19,The Sucker
45,The Fockwerds
50,Farter Spiper
59,toe Cock
61,Hey Dick
64,Fockwark
71,Ante Cunt
77,Green Facks
110,Hallal Butz
117,FucksiT
